In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
os.chdir('C:\\Users\\juanm\\OneDrive\\Escritorio\\pythonProject\\CoderSQL\\csvs\\originales')

In [3]:
books = pd.read_csv('books_data.csv')
ratings = pd.read_csv('Books_rating.csv')

In [ ]:
'''Datos originilales
Books (212404,10)
ratings(30000000,10)'''

In [4]:
#Aca va el proceso de normalizacion de las tablas
#Eliminar columnas
books = books.drop(columns=['image','previewLink','infoLink','ratingsCount', 'description'])
ratings = ratings.drop(columns=['review/helpfulness','review/time', 'review/summary', 'review/text'])
#merge de ambas columnas, a fin de eliminar valores y no dejar datos huerfanos
merged_df = pd.merge(books,ratings, on='Title')
merged_df = merged_df.dropna(subset=['Title']) #elimino filas donde 'Title' es NaN value
merged_df = merged_df.dropna(subset=['User_id', 'profileName'], how='all') #Elimino Nan Values relacionado con usuarios
merged_df = merged_df.dropna(subset=['profileName']) #Elimino Nan Values relacionado con usuarios
merged_df = merged_df.dropna(subset=['publisher', 'publishedDate', 'categories']) #elimino mas Nan values
merged_df = merged_df.dropna(subset=['authors'])#elimino mas Nan values

In [ ]:
merged_df.columns

In [5]:
'''CASO COLUMNA PRICE
Existe una cantidad enorme de Nan Values, pero algunos son recuperables, ya que segun el nro de ID, algunos books tiene precio,
entonces,procedo a completar esos valores que se pueden rellenar'''

# Calcular el valor promedio de Prices para cada book_id
avg_prices = merged_df.groupby('Id')['Price'].mean()

# Rellenar los valores NaN en Prices con el valor promedio para el book_id correspondiente
merged_df['Price'] = merged_df.apply(lambda row: avg_prices[row['Id']] if pd.isna(row['Price']) else row['Price'], axis=1)

#los prices que siguen quedando como Nan, se descartan
merged_df = merged_df.dropna(subset=['Price'])#elimino mas Nan values

'''De esta manera, queda un dataset con 306450 filas por 11 columnas
al cual hay que dividir en 5 tablas'''

'De esta manera, queda un dataset con 337447 filas por 11 columnas\nal cual hay que dividir en 5 tablas'

In [6]:
# genero nros de id desde 1
def add_factorize_id_column(df, column_name, new_column_name):
    df[new_column_name] = pd.factorize(df[column_name])[0] + 1
    return df

merged_df = add_factorize_id_column(merged_df, 'Title', 'id_book')
merged_df = add_factorize_id_column(merged_df, 'User_id', 'id_user')
merged_df = add_factorize_id_column(merged_df, 'categories', 'id_categories')

#Elimino mas columnas que no sirven
merged_df = merged_df.drop(columns=['Id', 'User_id'])

In [7]:
#Limpio las columnas de caracteres no deseados
def clean_column(col):
    col = col.str.replace('"', '').str.replace("'", '')
    col = col.str.strip('[]')
    col = col.str.replace('"', '').str.replace("'", '')
    return col

merged_df[['profileName', 'authors', 'categories']] = merged_df[['profileName', 'authors', 'categories']].apply(clean_column)

# Extrae solo el año de la columna 'publishedDate'
merged_df['publishedDate'] = merged_df['publishedDate'].str.extract(r'\b(\d{4})\b')

#cambio el nombre de las columnas
merged_df = merged_df.rename(columns={'Title': 'title', 'publishedDate': 'published_year', 'Price': 'price',
                                      'profileName': 'profile_name', 'review/score': 'score'})

#reinicio el indice
merged_df = merged_df.reset_index(drop=True)

In [35]:
#Separar merged_df en 5 tablas diferentes
def create_table(df, cols, drop_duplicates=None):
    table = df[cols]
    if drop_duplicates:
        table = table.drop_duplicates(subset=drop_duplicates)
    table = table.reset_index(drop=True)
    return table

books = create_table(merged_df, ['id_book', 'title', 'authors', 'publisher', 'published_year'], 'id_book')
reviews = create_table(merged_df, ['id_user', 'id_book', 'score'])
categories = create_table(merged_df, ['id_categories', 'categories'], 'id_categories')
prices = create_table(merged_df, ['id_book', 'price'], 'id_book')
users = create_table(merged_df, ['id_user', 'profile_name'], 'id_user')


In [36]:
#Dimensiones de las tablas una vez finalizado el proceso
print('books: ', books.shape)
print('reviews: ', reviews.shape)
print('categories: ', categories.shape)
print('prices: ', prices.shape)
print('users: ', users.shape)

books:  (33913, 5)
reviews:  (306450, 3)
categories:  (942, 2)
prices:  (33913, 2)
users:  (236353, 2)


In [41]:
reviews

,id_user,id_book,score
0,1,1,1.0
1,2,1,4.0
2,3,1,1.0
3,4,1,5.0
4,5,1,5.0
...,...,...,...
306445,236351,33912,4.0
306446,55555,33913,1.0
306447,3597,33913,4.0
306448,236352,33913,3.0


In [11]:
merged_df

,title,authors,publisher,published_year,categories,price,profile_name,score,id_book,id_user,id_categories
0,Whispers of the Wicked Saints,Veronica Haddon,iUniverse,2005,Fiction,10.95,Book Reader,1.0,1,1,1
1,Whispers of the Wicked Saints,Veronica Haddon,iUniverse,2005,Fiction,10.95,V. Powell,4.0,1,2,1
2,Whispers of the Wicked Saints,Veronica Haddon,iUniverse,2005,Fiction,10.95,LoveToRead Actually Read Books,1.0,1,3,1
3,Whispers of the Wicked Saints,Veronica Haddon,iUniverse,2005,Fiction,10.95,Clara,5.0,1,4,1
4,Whispers of the Wicked Saints,Veronica Haddon,iUniverse,2005,Fiction,10.95,Tonya,5.0,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...
306445,The Orphan Of Ellis Island (Time Travel Advent...,Elvira Woodruff,Scholastic Paperbacks,2000,Juvenile Fiction,3.32,jwinter,4.0,33912,236351,26
306446,The Autograph Man,Zadie Smith,Vintage,2003,Fiction,34.99,maryzeus maryzzz,1.0,33913,55555,1
306447,The Autograph Man,Zadie Smith,Vintage,2003,Fiction,34.99,Matko Vladanovic,4.0,33913,3597,1
306448,The Autograph Man,Zadie Smith,Vintage,2003,Fiction,34.99,tyuro,3.0,33913,236352,1


In [ ]:
#herramientas de consulta:
null_counts = merged_df.isnull().sum()
print(null_counts) #ver cuantos Nan values hay en cada columna
print('----------------')
print(merged_df.nunique())#Ver cuantos valores unicos hay por columna
print('----------------')
print(merged_df['Price'].describe())
print('----------------')
print(merged_df.shape)
print('----------------')

'''Use este codigo para ver cuantos prices pueden completarse con valores ya existentes
def has_mixed_prices(group):
    has_nan = np.isnan(group['Price']).any()
    has_number = np.isfinite(group['Price']).any()
    return has_nan and has_number

has_mixed = merged_df.groupby('id_book').apply(has_mixed_prices)
book_ids_with_mixed_prices = has_mixed[has_mixed].index.to_list()
print("Book IDs con precios mixtos:", book_ids_with_mixed_prices)'''

In [38]:
os.chdir('C:\\Users\\juanm\\OneDrive\\Escritorio\\pythonProject\\CoderSQL\\csvs\\normalizados')

In [42]:
books.to_csv('books.csv', index=False)
reviews.to_csv('reviews.csv', index=False)
categories.to_csv('categories.csv', index=False)
prices.to_csv('prices.csv', index=False)
users.to_csv('users.csv', index=False)
